In [1]:
import os

In [4]:
os.getcwd()

'/Users/vicky/Dev/optimizely/scripts'

In [55]:
import json
import requests
import popelines
import os
from datetime import datetime, timedelta
from main import fix_values, populating_vals, flatten, flatten_dupe_vals

In [56]:
from generate_original import read_endpoint, generate_projects
# from generate_original import generate_experiments, generate_results

In [57]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

In [58]:
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)


In [59]:
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

# Update project table

In [ ]:
# idea is to:
# GET [project_id, last_modified_ts]
# if new_project_id is not in {project_id} set: add to it
# if new_project_id is in {project_id} set: 
    # if last_modified_ts > bq.last_modified_ts: add [project_id, project_props, last_modified_ts(new), upload_ts]
    # else: do nothing

In [60]:
project_endpoint = 'https://api.optimizely.com/v2/projects'
all_projects = generate_projects(project_endpoint, headers)

In [61]:
all_projects

[{'account_id': 8725961507,
  'confidence_threshold': 0.9,
  'created': '2018-01-04T20:50:13.101130Z',
  'id': 9965963792,
  'is_classic': False,
  'last_modified': '2019-02-25T16:51:16.920893Z',
  'name': 'Free Trial',
  'platform': 'web',
  'sdks': [],
  'status': 'active',
  'web_snippet': {'code_revision': 210,
   'enable_force_variation': False,
   'exclude_disabled_experiments': False,
   'exclude_names': True,
   'include_jquery': False,
   'ip_anonymization': False,
   'ip_filter': '^(70\\.166\\.203\\.(?:1(?:2[8-9]|[3-8][0-9]|9[0-1])))|(174\\.67\\.125\\.(?:[0-9]|[1-9][0-9]|1(?:[0-1][0-9]|2[0-7])))|(70\\.166\\.189\\.(?:[0-7]))|(70\\.166\\.189\\.(?:3[2-9]|[4-5][0-9]|6[0-3]))|(174\\.78\\.90\\.(?:4[8-9]|5[0-5]))|(174\\.78\\.95\\.(?:5[6-9]))|(76\\.217\\.119\\.(?:2(?:2[4-9]|3[0-1])))|(12\\.196\\.29\\.(?:2(?:4[8-9]|5[0-5])))|(50\\.192\\.58\\.(?:2(?:0[8-9]|1[0-1])))|(76\\.250\\.184\\.(?:8[0-7]))|(12\\.133\\.142\\.(?:4[0-7]))|(67\\.133\\.44\\.(?:[8-9]|1[0-5]))$',
   'js_file_size': 8025

In [ ]:
check_project = []
for project in all_projects:
    check_project.append({'project_id': project['id'], 'last_modified': project['last_modified']})


In [ ]:
datetime.strptime(check_project[-1]['last_modified'], '%Y-%m-%dT%H:%M:%S.%fz')

In [ ]:
# this returns the local time for last upload timestamp of projects
# checking the last time this job ran
last_upload_ts = pope.find_last_entry('projects', 'upload_ts') + timedelta(hours=-8)

In [ ]:
last_upload_ts.replace(tzinfo=None)

In [ ]:
# compare each of the project's last_modified timestamp to the above last upload_ts
# select only if last_modified > upload_ts
update_project_id = []
for project in all_projects:
    if datetime.strptime(project['last_modified'], '%Y-%m-%dT%H:%M:%S.%fz') > last_upload_ts.replace(tzinfo=None):
        update_project_id.append(project['id'])

In [ ]:
# endpoints
# single_project_endpoint = f'https://api.optimizely.com/v2/projects/{project_id}'
# experiment_endpoint = 'https://api.optimizely.com/v2/experiments'

In [ ]:
update_project_id


In [ ]:
for project_id in update_project_id:
    single_project_endpoint = f'https://api.optimizely.com/v2/projects/{project_id}'
    project_info = generate_projects(single_project_endpoint, headers)
    project_info['upload_ts'] = str(datetime.utcnow())

In [ ]:
project_info

# Update experiment table

In [ ]:
os.system('say "completed"')

In [ ]:
datetime.utcnow() 

# Update results table

# Debug script

In [1]:
import json
import requests
import popelines
import os
from datetime import datetime, timedelta
from main import fix_values, populating_vals, flatten, flatten_dupe_vals
from generate_original import read_endpoint, generate_projects


In [62]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')

pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

In [5]:
# this function checks the last time a job was run on bq,
# returns the last-run timestamp for the table
def check_bq_ts(table_name, ts_col_name):
    last_upload_ts = pope.find_last_entry(table_name, ts_col_name) + timedelta(hours=-8)
    return last_upload_ts


In [43]:
# this function should get timestamp for each entity (project, experiment, or result), from API,
# returns only those with a timestamp that is later than the benchmark, benchmark_ts
def check_api_ts(api_path, table, ts_col, benchmark_ts, existing_id):
    
    # compare each of the project's last_modified timestamp to the above last upload_ts
    # select only if last_modified > upload_ts
    updating_entity_id = []
    if table == 'project':
        all_existing = generate_projects(api_path, headers)
    # elif table == 'experiment':
    #     all_existing = generate_experiment(api_path, headers)
    else:
        pass

    for entity in all_existing:
        entity_ts = datetime.strptime(entity[ts_col], '%Y-%m-%dT%H:%M:%S.%fz') + timedelta(hours=-8)
        if  entity_ts > benchmark_ts:
            updating_entity_id.append(entity['id'])
        elif entity['id'] not in existing_id:
            updating_entity_id.append(entity['id'])

    return updating_entity_id



In [7]:
# this function gathers all new entity dictionaries in a list, returns a format that is friendly to popelines
def generate_new_entity(id_list, api_path, table):
    new_json = []
    for entity_id in id_list:
        if table == 'project':
            entity_info = generate_projects(f'{api_path}/{entity_id}', headers)
            entity_info['upload_ts'] = str(datetime.utcnow())

        # need to add experiment table
        else:
            pass

        new_json.append(entity_info)

    return new_json

In [63]:
project_endpoint = 'https://api.optimizely.com/v2/projects'


In [64]:
# get the timestamp from bq for project table, at which the job was most recently run
last_upload_ts = check_bq_ts('projects', 'upload_ts')

In [65]:
last_upload_ts

datetime.datetime(2019, 2, 25, 14, 35, 11, 630901, tzinfo=<UTC>)

In [26]:
project_query = open('existing_projects.sql').read()

In [44]:
existing_projects = []
for result in pope.bq_query(project_query):
    existing_projects.append(result[0])

In [45]:
existing_projects

[10427612860,
 9965963792,
 10347872424,
 10429921250,
 10849384554,
 10057253317,
 13655420480]

In [66]:
new_list = check_api_ts(api_path=project_endpoint, table='project', ts_col='last_modified', benchmark_ts=last_upload_ts.replace(tzinfo=None), existing_id=existing_projects)


In [67]:
new_list

[]

In [14]:
benchmark_ts = last_upload_ts

In [15]:
benchmark_ts

datetime.datetime(2019, 2, 21, 21, 31, 44, 48589, tzinfo=<UTC>)

In [53]:
len(new_list)

6

In [48]:
generate_new_entity(id_list=new_list, api_path='https://api.optimizely.com/v2/projects', table='project')

[{'account_id': 8725961507,
  'confidence_threshold': 0.9,
  'created': '2018-01-04T20:50:13.101130Z',
  'id': 9965963792,
  'is_classic': False,
  'last_modified': '2019-02-25T16:51:16.920893Z',
  'name': 'Free Trial',
  'platform': 'web',
  'sdks': [],
  'status': 'active',
  'upload_ts': '2019-02-25 22:34:26.472794',
  'web_snippet': {'code_revision': 210,
   'enable_force_variation': False,
   'exclude_disabled_experiments': False,
   'exclude_names': True,
   'include_jquery': False,
   'ip_anonymization': False,
   'ip_filter': '^(70\\.166\\.203\\.(?:1(?:2[8-9]|[3-8][0-9]|9[0-1])))|(174\\.67\\.125\\.(?:[0-9]|[1-9][0-9]|1(?:[0-1][0-9]|2[0-7])))|(70\\.166\\.189\\.(?:[0-7]))|(70\\.166\\.189\\.(?:3[2-9]|[4-5][0-9]|6[0-3]))|(174\\.78\\.90\\.(?:4[8-9]|5[0-5]))|(174\\.78\\.95\\.(?:5[6-9]))|(76\\.217\\.119\\.(?:2(?:2[4-9]|3[0-1])))|(12\\.196\\.29\\.(?:2(?:4[8-9]|5[0-5])))|(50\\.192\\.58\\.(?:2(?:0[8-9]|1[0-1])))|(76\\.250\\.184\\.(?:8[0-7]))|(12\\.133\\.142\\.(?:4[0-7]))|(67\\.133\\.44\\

In [50]:
# get projects with last_modified timestamps that are later than the previous ts
# getting ready to upload
updating_projects = check_api_ts(api_path=project_endpoint, table='project', ts_col='last_modified', benchmark_ts=last_upload_ts.replace(tzinfo=None), existing_id=existing_projects)
updating_projects_json = generate_new_entity(id_list=updating_projects, api_path='https://api.optimizely.com/v2/projects', table='project')


In [52]:
len(updating_projects_json)

6

In [54]:
pope.write_to_json(file_name='../uploads/update_projects.json', jayson=updating_projects_json, mode='w')
pope.write_to_bq(table_name='projects', file_name='../uploads/update_projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)



INFO:root:Writing projects to BQ from file ../uploads/update_projects.json


2019-02-25 14:36:01,548 - root - Writing projects to BQ from file ../uploads/update_projects.json
